In [40]:
import load_data as ld
from biotransformers import BioTransformers
from datetime import datetime

Loader = ld.DataLoader()

train_df, val_df, test_df = Loader.load_all_three_dfs(max_size=1000)
train_input_seqs, val_input_seqs, test_input_seqs = map(Loader.numpy_input_sequence, (train_df, val_df, test_df))

In [4]:
esm_embedder = BioTransformers('esm1_t6_43M_UR50S')

Downloading:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

Some weights of the model checkpoint at Rostlab/prot_bert were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [41]:
start_time = datetime.now()
esm_embed_arrays = esm_embedder.compute_embeddings(train_input_seqs)["cls"]
end_time = datetime.now()
diff = end_time - start_time
mins = int(diff.seconds / 60)
print(mins)

 69%|██████▉   | 691/1000 [35:44<15:58,  3.10s/it]


KeyboardInterrupt: 

In [39]:
train_input_seqs

array(['SAKRRLYSLIQNATEPYAFEPDLAVNLDIADLINQTGGNLPREAAFAIVRKVNDRNPTVAYLALNLLDICVKNCGYAFRLQIASKEFLNELVRRFPERPPSRLNKIQVMILSLIEEWRKTICRVDRYKEDLGFIRDMHRLLSYKG',
       'LDLCIEEINRSIESISTLYFKPPGIFHNAVVHDQQGRGYSSIITKLIRDCNPKEELSLFKMDKQERIHTRKDGKRGNLDYLVERDAHLRRNRHIGIPDEKPIIQVPKEFYLKQHDEALAEKRKLVKGYNFDMESGVNSSGSNVYDLLLSKFKDEEVVSLIHALQNGSVLMGSEDGESRRKTMFVEDFPTEAILQVLHEVSTQWPLAEYKQAYSQHLNDYSNFNSEIELLRGEIQLQEEQLHLHKVNSSSSHVVARLIEKERRDIDALEKELIRQ',
       'AELKVLQAQIHPHFLFNALNTISSLIRTRPEKARELLAKLASFFRYSLRRDDNPIPLAEELAHVDSYLAIEEARFGNKL',
       'KCLDILYVAIQSIRVLTPTLRSLMYLVLNSLDHPHDSVHILALKTLCVYCILDKELAKKHIMIFFYQFSLEQENQEIWVISLRAIFDLLLVYGLEYFDIMQTPENNSVRNRSEKTRSLYTHEDSTHEDISISKRTEIEEGPCNFIKILTGLLTNANQDLRTIAAEGLCKLLLNRRINSSNLVSRLIIMCYNPVNVDDIYLRQCLSAFFDCFVGRVSDAQEMLENAYFLTLQVLCNAPDISPLREINAYHVSRFILGLTRRGCQKSSGQTFYTHNNLAFAILAEILNPKSKIDQEILIKSLTDLCI',
       'SCVQYTSCELCLGSRDPHCGWCVLHSICSRQDACERAEEPQRFASDLLQCV',
       'KRQRALEELYSKQEVPKSTYDEMRRKLDSEMLKVKDDYIKVKNSLKARLNEVEDQLAQIEKAIMSVKMSYISAELSEASYKNS

In [ ]:
start_time = datetime.now().strftime("%H:%M:%S")
protbert_embed_arrays = protbert_embedder.compute_embeddings(train_input_seqs)["cls"]
end_time = datetime.now().strftime("%H:%M:%S")